# Global Alignment

Name: Yanfang Guo           
Department : MACS       
Email: Yanfang.Guo@vub.be         



## Using Matrix to represent the BLOSOM matrix

In [1]:
class Matrix(object):
    def __init__(self,file):
        f = open(file)
        self.matrix ={}
        # c is the name of the column
        c = f.readline().split()
        for i in f:
            r = i.split()
            rowname = r.pop(0)
            t = {}
            for j in range(len(c)):
                t[c[j]] = int(r[j])
            self.matrix[rowname] = t

    def get(self,row,col):
        return self.matrix[row][col]

    def print(self):
        for key in self.matrix:
            for key2 in self.matrix[key]:

                print("{0:3d}".format(self.matrix[key][key2]),end="")

            print()

mat1 = Matrix("BLOSUM62.txt")
mat1.print()
    

  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4
 -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4
 -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4
 -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4
  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4
 -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4
 -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4
  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4
 -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4
 -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4
 -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4
 -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4
 -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4
 -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2

    

## Define I and E 
if we do not use openning gap penalty, simply set I=E

In [2]:
I = -6
E = -4

## read fasta file

In [3]:
def read_fasta(filename):
    f = open(filename)
    proteins = []
    d = f.read().split(">")

    # deal with the  "" before the first ""
    d.pop(0)
    for i in d:
        t = i.splitlines()
        # get rid of the line of the protein name
        t.pop(0)
        s = "".join(t)
        proteins.append(s)
    return proteins
seq_list = read_fasta("WW-sequence.fasta")
seq1 =seq_list[0]
seq2 =seq_list[1]

print(seq1)
print(seq2)

VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP


## Here we initialize the direc_mat 
- direc_mat is a i*j matrix, and the element of the direc_mat is a list with the length of 5.
- such as
 - t =[0,1,0,I,E]    
  t[0] t[1] t[2] indicates the direction of traceback.    
  t[3] t[4] use to deal with the gap penalty.     
  if t[0]=1 and t[3] is the first element of a gap, then t[3]=I(opening gap penalty), else t[3]=E.    
  the same with t[4]

In [4]:
direc_mat = []
# print_mat3, the function to print the direc_mat in a formative way.
def print_mat3(alist,n=3):
    for i in alist:
        for j in i:
            print(j[0:n],end='')
        print()

for i in range(len(seq1) + 1):
    # default, opening gap
    direc_mat.append([[0, 0, 0,I,I] for i in range(len(seq2) + 1)])


## Score_mat

In [5]:
score_mat = []
for i in range(len(seq1) + 1):
    score_mat.append([0 for i in range(len(seq2) + 1)])
    
# nicely output the score_mat
def print_mat2(alist):
    for i in alist:
        for j in i:
            print('{0:4d}'.format(j), end=" ")
        print()


## initialize the score_mat and direc_mat


In [6]:
score_mat[0][0] = 0

direc_mat[0][1] = [0,1,0,I,E]
score_mat[0][1]=I
direc_mat[1][0]=[1,0,0,E,I]
score_mat[1][0]=I

for i in range(2, len(seq2)+1):
    direc_mat[0][i] = [0,1,0,I,E]
    score_mat[0][i] = score_mat[0][i-1] + direc_mat[0][i-1][4]
for i in range(2, len(seq1)+1):
    direc_mat[i][0] = [1,0,0,E,I]
    score_mat[i][0] = score_mat[i-1][0]+ direc_mat[i-1][0][3]
    
#print_mat2(score_mat)

## Calculate the score


In [7]:
def score(mat):
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):

            t1 = score_mat[i - 1][j] + direc_mat[i-1][j][3]

            t2 = score_mat[i][j - 1] +direc_mat[i][j-1][4]

            t3 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])

            maxscore = max(t1, t2, t3)
            score_mat[i][j] = maxscore
            if (t1 == maxscore):
                direc_mat[i][j][1] = 1
                # affine gap
                direc_mat[i][j][3] = E
            if (t2 == maxscore):
                direc_mat[i][j][0] = 1
                direc_mat[i][j][4] = E
            if (t3 == maxscore):
                direc_mat[i][j][2] = 1
    return score_mat[len(seq1)][len(seq2)]
print(score())

TypeError: score() missing 1 required positional argument: 'mat'

## Trace back and print out the route

In [ ]:
def traceback():
    path_pair = []
    path_up = []
    path_down = []
    mark1 =[]
    mark2 =[]


    i = len(seq1)
    j = len(seq2)


    while (i > 0 and j > 0):
        tlist = direc_mat[i][j]
        if tlist[0] == 1:
            path_up.insert(0, '-')
            mark1.insert(0,' ')
            mark2.insert(0,' ')
            j = j - 1
            path_down.insert(0, seq2[j])
        elif tlist[1] == 1:
            path_down.insert(0, '-')
            mark1.insert(0, ' ')
            mark2.insert(0, ' ')
            i = i - 1
            path_up.insert(0, seq1[i])
        elif tlist[2] == 1:
            i = i - 1
            j = j - 1
            if seq1[i]==seq2[j]:
                mark1.insert(0,'.')
                mark2.insert(0,'.')
            elif mat1.get(seq1[i],seq2[j])>0:
                mark1.insert(0,' ')
                mark2.insert(0,'.')
            else:
                mark1.insert(0,' ')
                mark2.insert(0,' ')
            path_up.insert(0, seq1[i])
            path_down.insert(0, seq2[j])
    path_pair.append([path_up, mark1,mark2,path_down])
    return path_pair

def print_pathpair(alist):
    for path in alist:
        path_u = ''.join(path[0])
        path_d = ''.join(path[3])
        mark1=''.join(path[1])
        mark2 = ''.join(path[2])

        print(path_u)
        print(mark1)
        print(mark2)
        print(path_d)

print_pathpair(traceback())